In [13]:
import sklearn as sk
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

print('numpy version:', np.__version__)
print('pandas version:', pd.__version__)
print('scikit-learn version:', sk.__version__)
print('matplotlib version:', matplotlib.__version__)

%matplotlib inline

numpy version: 1.12.1
pandas version: 0.20.1
scikit-learn version: 0.18.1
matplotlib version: 2.0.2


In [14]:
datafolder = 'data/'
filename = 'dataset_informa.csv'
df = pd.read_csv(datafolder + filename, sep=',', low_memory=False, encoding = 'ISO-8859-1')
print(df.shape)
df.head()

(190570, 49)


,AGENCY_ID,PRIMARY_AGENCY_ID,PROD_ABBR,PROD_LINE,STATE_ABBR,STAT_PROFILE_DATE_YEAR,RETENTION_POLY_QTY,POLY_INFORCE_QTY,PREV_POLY_INFORCE_QTY,NB_WRTN_PREM_AMT,...,PL_BOUND_CT_ELINKS,PL_QUO_CT_ELINKS,PL_BOUND_CT_PLRANK,PL_QUO_CT_PLRANK,PL_BOUND_CT_eQTte,PL_QUO_CT_eQTte,PL_BOUND_CT_APPLIED,PL_QUO_CT_APPLIED,PL_BOUND_CT_TRANSACTNOW,PL_QUO_CT_TRANSACTNOW
0,3,3,BOILERMACH,CL,IN,2005,0,0,0,40.0,...,0,0,0,103,50,288,0,0,0,0
1,3,3,BOILERMACH,CL,IN,2006,0,0,0,151.0,...,0,0,0,103,50,288,0,0,0,0
2,3,3,BOILERMACH,CL,IN,2007,0,0,0,40.0,...,0,0,0,103,50,288,0,0,0,0
3,3,3,BOILERMACH,CL,IN,2008,0,0,0,69.0,...,0,0,0,103,50,288,0,0,0,0
4,3,3,BOILERMACH,CL,IN,2009,0,0,0,28.0,...,0,0,0,103,50,288,0,0,0,0


In [15]:
# Primary_Agency_ID: When this column data is missing/99999 we are going to replace it with the Agency_ID. 
# We can not find any way to associate a primary agency with the agency in question so will replace it with its ID.
df['PRIMARY_AGENCY_ID'] = np.where(df['PRIMARY_AGENCY_ID'] == 0, df['PRIMARY_AGENCY_ID'], df['AGENCY_ID'])

# Prev_Poly_Inforce_QTY: Contains 99999 values. 
# These values mean there are no previous years or this year did not have data (CL).
# These values will be replaces by 0
df['PREV_POLY_INFORCE_QTY'] = df['PREV_POLY_INFORCE_QTY'].replace(99999,0)

# Agency_appointment_year: 5000 missing data, irrelevant column, will be replaced with average.
mean = df['AGENCY_APPOINTMENT_YEAR'].mean()
df['AGENCY_APPOINTMENT_YEAR'].replace(99999, mean) 

# Active_Producers, Max_age and min_age: 
# If the amount of producers is not set (99999) this can be set to 0, this also goes for max and min age.
df['ACTIVE_PRODUCERS'].replace(99999, 0) 
df['MAX_AGE'].replace(99999, 0) 
df['MIN_AGE'].replace(99999, 0) 

# (VENDOR)_START_YEAR: Values are varied between 1994 and 2015. 
# When the value is missing (99999) the vendor was not used. 
# All 99999 values can be replaced with 0.
df['PL_START_YEAR'].replace(99999, 0)
df['CL_START_YEAR'].replace(99999, 0)

# (VENDOR)_END _YEAR: The year the agency stopped using the vendor. 
# This can either mean the vendor is never used or the vendor is still in use. 
# If the vendor has never been used we can replace the value with 0, if the vendor is still in use we can replace it with 1.
df['CL_END_YEAR'] = np.where(df['CL_START_YEAR'] == 0, 0, 1)
df['PL_END_YEAR'] = np.where(df['PL_START_YEAR'] == 0, 0, 1)


In [16]:
# split PL and CL datasets
df_cl = df[df.PROD_LINE=='CL']
df_pl = df[df.PROD_LINE=='PL']

# remove irrelevant columns from splits
df_cl = df_cl.filter(regex='^(?!PL_)\w+', axis=1)
df_pl = df_pl.filter(regex='^(?!CL_)\w+', axis=1)

In [17]:
# write dataframes to csv files
df_cl.to_csv('cleaned_CL_'+filename)
df_pl.to_csv('cleaned_PL_'+filename)